In [1]:
#this combines the files together in the directories
import pandas as pd
import numpy as np
import pathlib2 as pl2

def combine_files_together():
    base_path = "../data/"
    elms_path = "ELMS"
    lmc_cup_path = "LeMansCup"
    fiawec_path = "FIAWEC"
    imsa_path = "IMSA"

    file_paths = [fiawec_path, elms_path, lmc_cup_path, imsa_path]
    #file_paths = [fiawec_path, imsa_path]
    #file_paths = [lmc_cup_path]
    #combine the dfs together
    for f in file_paths:
        curr_dir = base_path + f
        full_df = pd.DataFrame()
        current_circuit = ""
        ps = pl2.Path(curr_dir)
        
        rows = (pd.read_csv(p, index_col=0, dtype=str) for p in ps.glob('*.csv'))
        full_df = pd.concat(rows)
        full_csv_loc = f + "_full_data.csv"
        full_df = full_df.dropna(axis=1, how = "all")
        full_df = full_df.reset_index(drop=True)
        full_df.columns = full_df.columns.str.strip()
        full_df.columns = full_df.columns.str.lower()
        full_df.to_csv(full_csv_loc)
combine_files_together()

In [3]:
#load all the combined dfs
def load_all_dfs():
    fia_wec_df = pd.read_csv('FIAWEC_full_data.csv', index_col = 0, dtype=str)
    imsa_df = pd.read_csv('IMSA_full_data.csv', index_col = 0,dtype=str)
    elms_df = pd.read_csv('ELMS_full_data.csv', index_col = 0,dtype=str)
    lmc_df = pd.read_csv('LeMansCup_full_data.csv', index_col = 0,dtype=str)
    full_df = pd.concat([fia_wec_df, imsa_df, elms_df, lmc_df])
    #full_df = pd.concat([fia_wec_df, imsa_df])
    full_df = full_df.reset_index(drop=True)
    return full_df
full_df = load_all_dfs()
full_df.to_csv('full_data_2023.csv')

In [4]:
full_df

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,vehicle,championship,session_type,season,circuit,round,class_x,team_x,class_y,team_y
0,10,1,1,2:41.583,0,NaN,1:19.587,0,39.859,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,NaN,NaN,NaN,NaN
1,10,1,2,1:44.629,0,NaN,34.289,0,33.328,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,NaN,NaN,NaN,NaN
2,10,1,3,1:40.545,0,NaN,31.160,0,31.766,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,NaN,NaN,NaN,NaN
3,10,1,4,1:37.171,0,NaN,30.593,0,31.194,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,NaN,NaN,NaN,NaN
4,10,1,5,1:36.305,0,NaN,30.345,0,30.641,2,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136293,97,1,55,1:42.759,0,NaN,30.696,0,41.533,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,LMP3,COOL Racing,LMP3,COOL Racing
136294,97,1,56,1:44.681,0,NaN,30.698,0,42.828,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,LMP3,COOL Racing,LMP3,COOL Racing
136295,97,1,57,1:41.929,0,NaN,30.529,0,40.869,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,LMP3,COOL Racing,LMP3,COOL Racing
136296,97,1,58,1:41.954,0,NaN,30.400,0,41.071,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,LMP3,COOL Racing,LMP3,COOL Racing


In [5]:
full_df['key'] = full_df['season'].map(str) + "_" + full_df['championship'] + "_" + full_df['round'].map(str) \
        + "_" + full_df['session_type'] + "_" + \
                full_df['number'].map(str) +  "_" + \
                full_df['lap_number'].map(str)
full_df[full_df['number'] == '04']

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,championship,session_type,season,circuit,round,class_x,team_x,class_y,team_y,key
29672,04,1,1,2:38.902,0,NaN,1:10.174,0,56.622,0,...,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_2_PRACTICE 1_04_1
29673,04,1,2,1:45.464,0,NaN,25.912,0,48.562,0,...,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_2_PRACTICE 1_04_2
29674,04,1,3,2:14.649,0,B,25.452,0,50.423,0,...,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_2_PRACTICE 1_04_3
29675,04,1,4,6:32.378,0,NaN,5:06.872,0,52.816,0,...,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_2_PRACTICE 1_04_4
29676,04,1,5,1:46.784,0,NaN,25.468,0,48.199,0,...,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_2_PRACTICE 1_04_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98979,04,3,314,4:10.735,0,NaN,28.887,0,2:32.447,0,...,IMSA,RACE,2023,Sebring International Raceway,4,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_4_RACE_04_314
98980,04,3,315,3:33.173,0,NaN,26.060,0,2:13.028,0,...,IMSA,RACE,2023,Sebring International Raceway,4,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_4_RACE_04_315
98981,04,3,316,2:07.803,0,NaN,10.903,0,1:24.567,0,...,IMSA,RACE,2023,Sebring International Raceway,4,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_4_RACE_04_316
98982,04,3,317,1:56.264,0,NaN,10.718,0,1:13.323,0,...,IMSA,RACE,2023,Sebring International Raceway,4,LMP2,Crowdstrike Racing by APR,LMP2,Crowdstrike Racing by APR,2023_IMSA_4_RACE_04_317


In [6]:
full_df = full_df.drop_duplicates()

In [7]:
# no duplicates
df_counts = full_df.groupby('key').count()
df_counts[df_counts['number'] > 1]

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,vehicle,championship,session_type,season,circuit,round,class_x,team_x,class_y,team_y
key,,,,,,,,,,,,,,,,,,,,,


In [2]:
import numpy as np
def fix_columns(full_df):
    #these help fix the team, group, class
    full_df['fix_team'] = np.where(full_df['team'].isna(), np.where(full_df['team_x'].isna(), full_df['team_y'], full_df['team_x']),full_df['team'])
    full_df['fix_class'] = np.where(full_df['class'].isna(), np.where(full_df['class_x'].isna(), full_df['class_y'], full_df['class_x']),full_df['class'])
    full_df['fix_group'] = np.where(full_df['group'].isna(), np.where(full_df['group_x'].isna(), full_df['group_y'], full_df['group_x']),full_df['group'])

    full_df['team'] = full_df['fix_team']
    full_df['class'] = full_df['fix_class']
    full_df['group'] = full_df['fix_group']

    full_df_fixed_columns = full_df.drop(['team_x', 'team_y', 'fix_team', 'group_x', 'group_y', 'fix_group', 'class_x', 'class_y','fix_class'], axis=1)
    return full_df_fixed_columns
full_df_fixed_columns = fix_columns(full_df)

NameError: name 'full_df' is not defined

In [12]:
full_df_fixed_columns[full_df_fixed_columns['team'].isna()].groupby(['season','championship','round','session_type']).count()

,,,,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,group,team,manufacturer,flag_at_fl,s1_seconds,s2_seconds,s3_seconds,vehicle,circuit,key
season,championship,round,session_type,,,,,,,,,,,,,,,,,,,,,


In [13]:
#okay so the next thing to prob fix to drop porsche,ford,t6,t7, not used really for anything
#also fix s1,s2,s3, pit_time and convert to seconds
full_df_fixed_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136298 entries, 0 to 136297
Data columns (total 36 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   number                       136298 non-null  object
 1   driver_number                136298 non-null  object
 2   lap_number                   136298 non-null  object
 3   lap_time                     136298 non-null  object
 4   lap_improvement              136298 non-null  object
 5   crossing_finish_line_in_pit  11478 non-null   object
 6   s1                           136269 non-null  object
 7   s1_improvement               136298 non-null  object
 8   s2                           136155 non-null  object
 9   s2_improvement               136298 non-null  object
 10  s3                           136171 non-null  object
 11  s3_improvement               136298 non-null  object
 12  kph                          136298 non-null  object
 13  elapsed       

In [14]:
def fix_full_df_without_lm(full_df_fixed_columns):
    #drop the ones we don't need like the le mans specific ones
    full_df_without_lm = full_df_fixed_columns.drop(['porsche', 'ford', 't6', 't7'], axis=1)
    return full_df_without_lm
full_df_without_lm = fix_full_df_without_lm(full_df_fixed_columns)

KeyError: "['porsche' 'ford' 't6' 't7'] not found in axis"

In [18]:
full_df_without_large = full_df_fixed_columns.drop(['s1_large', 's2_large', 's3_large'], axis=1)
full_df_without_large.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136298 entries, 0 to 136297
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   number                       136298 non-null  object
 1   driver_number                136298 non-null  object
 2   lap_number                   136298 non-null  object
 3   lap_time                     136298 non-null  object
 4   lap_improvement              136298 non-null  object
 5   crossing_finish_line_in_pit  11478 non-null   object
 6   s1                           136269 non-null  object
 7   s1_improvement               136298 non-null  object
 8   s2                           136155 non-null  object
 9   s2_improvement               136298 non-null  object
 10  s3                           136171 non-null  object
 11  s3_improvement               136298 non-null  object
 12  kph                          136298 non-null  object
 13  elapsed       

In [19]:
#this we use to convert anything to ms
def convert_to_s(x):
    if(x == np.nan):
        return x
    hour = 0
    minutes = 0
    try:
        hour, minutes, second = x.split(":")
    except ValueError:
        hour = 0
        try:
            minutes, second = x.split(":")
        except ValueError:
            hour = 0
            minutes = 0
            second = x
    except AttributeError:
        print(x)
    hour_seconds = int(hour)*60*60
    if hour == 0 and minutes == 0:
        return x
    else:
        ms = 0
        try:
            seconds, ms = second.split(".")
        except ValueError:
            seconds = second
    minute_seconds = (int(minutes)*60 + int(seconds) + int(hour_seconds))*1000 
    total_ms = minute_seconds + int(ms)
    return total_ms/1000

#this uses the method to convert all of the large ones (h:m:s.ms to seconds)
def convert_df_cols_to_seconds(full_df):
    full_df['lap_time_seconds'] = full_df.lap_time.map(convert_to_s)
    full_df['s1_seconds'] = full_df.s1.map(convert_to_s)
    full_df['s2_seconds'] = full_df.s2.map(convert_to_s)
    full_df['s3_seconds'] = full_df.s3.map(convert_to_s)
    full_df['elapsed_seconds'] = full_df.elapsed.map(convert_to_s)
    full_df['pit_time_seconds'] = full_df.pit_time.map(convert_to_s)
    return full_df
full_df_fix_seconds = convert_df_cols_to_seconds(full_df_without_large)
full_df_fix_seconds 

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,vehicle,championship,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds
0,10,1,1,2:41.583,0,NaN,1:19.587,0,39.859,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_1,161.583,161.583,39.559
1,10,1,2,1:44.629,0,NaN,34.289,0,33.328,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_2,104.629,266.212,NaN
2,10,1,3,1:40.545,0,NaN,31.160,0,31.766,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_3,100.545,366.757,NaN
3,10,1,4,1:37.171,0,NaN,30.593,0,31.194,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_4,97.171,463.928,NaN
4,10,1,5,1:36.305,0,NaN,30.345,0,30.641,2,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_5,96.305,560.233,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136293,97,1,55,1:42.759,0,NaN,30.696,0,41.533,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_55,102.759,9917.958,NaN
136294,97,1,56,1:44.681,0,NaN,30.698,0,42.828,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_56,104.681,10022.639,NaN
136295,97,1,57,1:41.929,0,NaN,30.529,0,40.869,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_57,101.929,10124.568,NaN
136296,97,1,58,1:41.954,0,NaN,30.400,0,41.071,0,...,Ligier JS P320 - Nissan,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_58,101.954,10226.522,NaN


In [20]:
full_df_fix_seconds['session'] = full_df_fix_seconds['session_type']
full_df_fix_seconds

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,championship,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds,session
0,10,1,1,2:41.583,0,NaN,1:19.587,0,39.859,0,...,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_1,161.583,161.583,39.559,FREE PRACTICE 1
1,10,1,2,1:44.629,0,NaN,34.289,0,33.328,0,...,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_2,104.629,266.212,NaN,FREE PRACTICE 1
2,10,1,3,1:40.545,0,NaN,31.160,0,31.766,0,...,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_3,100.545,366.757,NaN,FREE PRACTICE 1
3,10,1,4,1:37.171,0,NaN,30.593,0,31.194,0,...,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_4,97.171,463.928,NaN,FREE PRACTICE 1
4,10,1,5,1:36.305,0,NaN,30.345,0,30.641,2,...,FIAWEC,FREE PRACTICE 1,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_5,96.305,560.233,NaN,FREE PRACTICE 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136293,97,1,55,1:42.759,0,NaN,30.696,0,41.533,0,...,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_55,102.759,9917.958,NaN,MORNING SESSION
136294,97,1,56,1:44.681,0,NaN,30.698,0,42.828,0,...,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_56,104.681,10022.639,NaN,MORNING SESSION
136295,97,1,57,1:41.929,0,NaN,30.529,0,40.869,0,...,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_57,101.929,10124.568,NaN,MORNING SESSION
136296,97,1,58,1:41.954,0,NaN,30.400,0,41.071,0,...,LeMansCup,MORNING SESSION,2023,OFFICIAL TEST - BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_58,101.954,10226.522,NaN,MORNING SESSION


In [21]:
def map_sessions(s):
    if ('QUALIFYING' in s):
        if ('RACE' in s):
            return 'RACE'
        return 'QUALIFYING'
    elif('SESSION' in s or 'TEST' in s):
        return 'TEST'
    elif ('PRACTICE' in s):
        return 'FREE PRACTICE'
    elif ('RACE' in s):
        return 'RACE'
    else:
        return s
#maps the sessions to session_types

full_df_fix_seconds['session_type'] = full_df_fix_seconds['session'].map(map_sessions)
full_df_test = full_df_fix_seconds.groupby(['session','session_type']).mean()
full_df_test

,,lap_time_seconds,elapsed_seconds,pit_time_seconds
session,session_type,,,
AFTERNOON SESSION,TEST,203.248068,6079.004699,642.766572
FREE PRACTICE 1,FREE PRACTICE,152.565648,2455.790495,242.306345
FREE PRACTICE 2,FREE PRACTICE,148.571781,2683.794254,240.033482
FREE PRACTICE 3,FREE PRACTICE,157.299640,1813.118943,171.646173
MORNING SESSION,TEST,195.159065,6973.271177,654.706393
NIGHT SESSION,TEST,168.747314,3066.287767,513.183895
PRACTICE 1,FREE PRACTICE,182.318096,2453.543034,329.933401
PRACTICE 2,FREE PRACTICE,158.107401,2913.284881,304.315433
PRACTICE 3,FREE PRACTICE,184.081983,3100.184866,306.329102


In [22]:
full_df_fix_seconds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136298 entries, 0 to 136297
Data columns (total 37 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   number                       136298 non-null  object 
 1   driver_number                136298 non-null  object 
 2   lap_number                   136298 non-null  object 
 3   lap_time                     136298 non-null  object 
 4   lap_improvement              136298 non-null  object 
 5   crossing_finish_line_in_pit  11478 non-null   object 
 6   s1                           136269 non-null  object 
 7   s1_improvement               136298 non-null  object 
 8   s2                           136155 non-null  object 
 9   s2_improvement               136298 non-null  object 
 10  s3                           136171 non-null  object 
 11  s3_improvement               136298 non-null  object 
 12  kph                          136298 non-null  object 
 13 

In [ ]:
full_df_fix_seconds[(full_df_fix_seconds['team'].isna())].to_csv('missing_teams.csv')
#I have to fix this
#2012, 2013, 2014 fiawec data is missing qualifying classifications so we have to use a race qualifcation to map properly
#first map 


#2012 elms 1 and 2012 fiawec 1 is missing in fp1/2, not sure why
full_df_missing_teams_fiawec = full_df_missing_teams[full_df_missing_teams['championship'] == 'FIAWEC']

In [26]:
full_df_fix_seconds_columns = full_df_fix_seconds.drop(['s1', 's2', 's3', 'lap_time', 'elapsed', 'pit_time'], axis=1)

In [27]:
full_df_fix_seconds_columns.to_csv('alkamel_data_full.csv', index=False)

In [28]:
import pandas as pd
full_df = pd.read_csv('alkamel_data_full.csv', dtype=str)
wanted_columns = ['event', 'circuit']
full_df['event'] = full_df['circuit']
full_df_group = full_df.groupby(['circuit','event']).mean().reset_index()
full_df_group[wanted_columns].to_csv('new_2022_events.csv',index=False)

c:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
events = pd.read_csv('new_2022_events.csv')
full_df_w_event = pd.merge(left = full_df, right = events, on = 'circuit')
full_df_w_event['event'] = full_df_w_event['event_x']
full_df_w_event['circuit'] = full_df_w_event['event_y']
full_df_w_event = full_df_w_event.drop(['event_x', 'event_y'], axis=1)
full_df_w_event.groupby(['circuit','event']).count()

number  \
circuit                        event                                         
AUTODROMO DO ALGARVE           AUTODROMO DO ALGARVE                  11852   
BARCELONA-CATALUNYA            BARCELONA-CATALUNYA                   12643   
                               OFFICIAL TEST - BARCELONA-CATALUNYA   10129   
Daytona International Speedway Daytona International Speedway        46352   
                               ROAR Before the 24                     8487   
Long Beach Street Circuit      Long Beach Street Circuit              4385   
SPA FRANCORCHAMPS              SPA FRANCORCHAMPS                      7637   
Sebring International Raceway  SEBRING                               10119   
                               Sebring February Test                  4941   
                               Sebring International Raceway         19753   

                                                                    driver_number  \
circuit                        event                                                
AUTODROMO DO ALGARVE           AUTODROMO DO ALGARVE                         11852   
BARCELONA-CATALUNYA            BARCELONA-CATALUNYA                          12643   
                               OFFICIAL TEST - BARCELONA-CATALUNYA          10129   
Daytona International Speedway Daytona International Speedway               46352   
                               ROAR Before the 24                            8487   
Long Beach Street Circuit      Long Beach Street Circuit                     4385   
SPA FRANCORCHAMPS              SPA FRANCORCHAMPS                             7637   
Sebring International Raceway  SEBRING                                      10119   
                               Sebring February Test                         4941   
                               Sebring International Raceway                19753   

                                                                    lap_number  \
circuit                        event                                             
AUTODROMO DO ALGARVE           AUTODROMO DO ALGARVE                      11852   
BARCELONA-CATALUNYA            BARCELONA-CATALUNYA                       12643   
                               OFFICIAL TEST - BARCELONA-CATALUNYA       10129   
Daytona International Speedway Daytona International Speedway            46352   
                               ROAR Before the 24                         8487   
Long Beach Street Circuit      Long Beach Street Circuit                  4385   
SPA FRANCORCHAMPS              SPA FRANCORCHAMPS                          7637   
Sebring International Raceway  SEBRING                                   10119   
                               Sebring February Test                      4941   
                               Sebring International Raceway             19753   

                                                                    lap_improvement  \
circuit                        event                                                  
AUTODROMO DO ALGARVE           AUTODROMO DO ALGARVE                           11852   
BARCELONA-CATALUNYA            BARCELONA-CATALUNYA                            12643   
                               OFFICIAL TEST - BARCELONA-CATALUNYA            10129   
Daytona International Speedway Daytona International Speedway                 46352   
                               ROAR Before the 24                              8487   
Long Beach Street Circuit      Long Beach Street Circuit                       4385   
SPA FRANCORCHAMPS              SPA FRANCORCHAMPS                               7637   
Sebring International Raceway  SEBRING                                        10119   
                               Sebring February Test                           4941   
                               Sebring International Raceway                  19753   

                                                                    crossing_fin

In [31]:
def remap_columns(full_df_w_event):
    full_df_w_event['driver_number'] = full_df_w_event['driver_number'].map(int)
    full_df_w_event['lap_number'] = full_df_w_event['lap_number'].map(int)
    full_df_w_event['lap_improvement'] = full_df_w_event['lap_improvement'].map(int)
    full_df_w_event['s1_improvement'] = full_df_w_event['s1_improvement'].map(int)
    full_df_w_event['s2_improvement'] = full_df_w_event['s2_improvement'].map(int)
    full_df_w_event['s3_improvement'] = full_df_w_event['s3_improvement'].map(int)
    full_df_w_event['kph'] = full_df_w_event['kph'].map(float)
    full_df_w_event['top_speed'] = full_df_w_event['top_speed'].map(float)
    full_df_w_event['round'] = full_df_w_event['round'].map(int)
    full_df_w_event['lap_time_seconds'] = full_df_w_event['lap_time_seconds'].map(float)
    full_df_w_event['s1_seconds'] = full_df_w_event['s1_seconds'].map(float)
    full_df_w_event['s2_seconds'] = full_df_w_event['s2_seconds'].map(float)
    full_df_w_event['s3_seconds'] = full_df_w_event['s3_seconds'].map(float)
    full_df_w_event['elapsed_seconds'] = full_df_w_event['elapsed_seconds'].map(float)
    full_df_w_event['pit_time_seconds'] = full_df_w_event['pit_time_seconds'].map(float)
    full_df_w_event['season'] = full_df_w_event['season'].map(str)
    return full_df_w_event
full_df_w_event_fixed_columns = remap_columns(full_df_w_event)
full_df_w_event_fixed_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136298 entries, 0 to 136297
Data columns (total 32 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   number                       136298 non-null  object 
 1   driver_number                136298 non-null  int64  
 2   lap_number                   136298 non-null  int64  
 3   lap_improvement              136298 non-null  int64  
 4   crossing_finish_line_in_pit  11478 non-null   object 
 5   s1_improvement               136298 non-null  int64  
 6   s2_improvement               136298 non-null  int64  
 7   s3_improvement               136298 non-null  int64  
 8   kph                          136298 non-null  float64
 9   hour                         136298 non-null  object 
 10  top_speed                    50927 non-null   float64
 11  driver_name                  136298 non-null  object 
 12  class                        136298 non-null  object 
 13 

In [32]:
full_df_w_event_non_2023 = full_df_w_event_fixed_columns[full_df_w_event_fixed_columns['season']!='2023']
full_df_w_event_non_2023.to_csv('alkamel_data_2022.csv')

In [34]:
full_df_w_event_2023 = full_df_w_event[full_df_w_event['season']=='2023']
full_df_w_event_2023

,number,driver_number,lap_number,lap_improvement,crossing_finish_line_in_pit,s1_improvement,s2_improvement,s3_improvement,kph,hour,...,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds,session,event
0,10,1,1,0,NaN,0,0,0,97.7,10:32:41.583,...,FREE PRACTICE,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_1,161.583,161.583,39.559,FREE PRACTICE 1,AUTODROMO DO ALGARVE
1,10,1,2,0,NaN,0,0,0,160.1,10:34:26.212,...,FREE PRACTICE,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_2,104.629,266.212,NaN,FREE PRACTICE 1,AUTODROMO DO ALGARVE
2,10,1,3,0,NaN,0,0,0,166.6,10:36:06.757,...,FREE PRACTICE,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_3,100.545,366.757,NaN,FREE PRACTICE 1,AUTODROMO DO ALGARVE
3,10,1,4,0,NaN,0,0,0,172.4,10:37:43.928,...,FREE PRACTICE,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_4,97.171,463.928,NaN,FREE PRACTICE 1,AUTODROMO DO ALGARVE
4,10,1,5,0,NaN,0,2,1,173.9,10:39:20.233,...,FREE PRACTICE,2023,AUTODROMO DO ALGARVE,2,2023_FIAWEC_2_FREE PRACTICE 1_10_5,96.305,560.233,NaN,FREE PRACTICE 1,AUTODROMO DO ALGARVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136293,97,1,55,0,NaN,0,0,0,163.2,11:45:17.958,...,TEST,2023,BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_55,102.759,9917.958,NaN,MORNING SESSION,OFFICIAL TEST - BARCELONA-CATALUNYA
136294,97,1,56,0,NaN,0,0,0,160.2,11:47:02.639,...,TEST,2023,BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_56,104.681,10022.639,NaN,MORNING SESSION,OFFICIAL TEST - BARCELONA-CATALUNYA
136295,97,1,57,0,NaN,0,0,0,164.5,11:48:44.568,...,TEST,2023,BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_57,101.929,10124.568,NaN,MORNING SESSION,OFFICIAL TEST - BARCELONA-CATALUNYA
136296,97,1,58,0,NaN,0,0,0,164.4,11:50:26.522,...,TEST,2023,BARCELONA-CATALUNYA,1,2023_LeMansCup_1_MORNING SESSION_97_58,101.954,10226.522,NaN,MORNING SESSION,OFFICIAL TEST - BARCELONA-CATALUNYA


In [36]:
full_df_w_event_2023.columns

Index(['number', 'driver_number', 'lap_number', 'lap_improvement',
       'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement',
       's3_improvement', 'kph', 'hour', 'top_speed', 'driver_name', 'class',
       'group', 'team', 'manufacturer', 'flag_at_fl', 's1_seconds',
       's2_seconds', 's3_seconds', 'vehicle', 'championship', 'session_type',
       'season', 'circuit', 'round', 'key', 'lap_time_seconds',
       'elapsed_seconds', 'pit_time_seconds', 'session', 'event'],
      dtype='object')

In [35]:
full_df_w_event_2023.to_csv('alkamel_data_2023.csv', index=False)

In [ ]:
full_df_w_event.columns